In [1]:
import os
import re
import pandas as pd

In [2]:
vsize = 100 # output vector size

In [3]:
df = pd.read_csv(os.path.join('data','raw','2020-08-24-listings-with-nlp.csv.gz'))
df.sample(2)

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,geometry,description_norm
4293,3791899.0,https://www.airbnb.com/rooms/3791899,20200824024753,2020-08-26,Stunning flat London-Fitzrovia,Bright stunning flat situated right in the tre...,"Close to the busy shopping of Oxford Circus, t...",https://a0.muscache.com/pictures/64097664/8ba4...,13665007.0,https://www.airbnb.com/users/show/13665007,...,9.0,NaN,f,1.0,1.0,0.0,0.0,2.59,POINT (-0.14003 51.51869),bright stun flat situate right trendy heart lo...
64322,40938752.0,https://www.airbnb.com/rooms/40938752,20200824024753,2020-08-26,City Riverview 1 Bedroom Loft Penthouse Apartment,NaN,"breathtaken riverview, canary wharf and city view",https://a0.muscache.com/pictures/1c7d5abd-f546...,303918107.0,https://www.airbnb.com/users/show/303918107,...,NaN,NaN,t,3.0,0.0,3.0,0.0,NaN,POINT (-0.06523999999999999 51.50466),NaN


In [4]:
df['dn'] = df['description_norm'].str.replace(r'[^\w\s]',' ').str.strip().astype(str)

In [5]:
df['dn'] = df.dn.apply(lambda x: ' '.join([word for word in x.split() if not re.search(r'\d', word)]))

In [7]:
import gensim
from nltk import word_tokenize
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

corpus = [word_tokenize(doc) for doc in df['dn'].values]
corpus = [
    TaggedDocument(words, ['d{}'.format(idx)])
    for idx, words in enumerate(corpus)
]

model = Doc2Vec(corpus, vector_size=vsize, min_count=15, workers=6, max_vocab_size=5*1e6)
model.save(os.path.join('data','clean',f"doc2vec-model-{vsize}.gensim"))

In [ ]:
model.docvecs.vectors_docs

In [8]:
maxv = 0
minv = 0
for i in model.docvecs.vectors_docs:
    if max(i) > maxv:
        maxv = max(i)
    if min(i) < minv:
        minv = min(i)

print(maxv)
print(minv)

0.6858632
-0.5827521


In [9]:
len(model.docvecs.vectors_docs.T)

100

In [10]:
w2v = pd.DataFrame.from_dict({'index':df.index, 'id':df.id})

i = 1
for v in model.docvecs.vectors_docs.T:
    w2v[f"WordVec {i}"] = v
    i += 1

w2v['id'] = w2v.id.astype('int')
w2v = w2v.set_index('index')
w2v.head()

,id,WordVec 1,WordVec 2,WordVec 3,WordVec 4,WordVec 5,WordVec 6,WordVec 7,WordVec 8,WordVec 9,...,WordVec 91,WordVec 92,WordVec 93,WordVec 94,WordVec 95,WordVec 96,WordVec 97,WordVec 98,WordVec 99,WordVec 100
index,,,,,,,,,,,,,,,,,,,,,
0,11551,0.008480,0.066643,-0.080251,0.207269,-0.096470,-0.040958,-0.073221,0.092468,-0.225067,...,-0.132938,-0.004312,-0.069694,0.008521,0.127871,-0.024403,-0.077414,0.000936,0.111261,-0.065022
1,13913,0.012211,0.100973,-0.157095,0.033285,-0.040324,0.056471,-0.138256,-0.078985,-0.139327,...,-0.055745,0.078799,-0.173785,-0.027293,0.120776,-0.025887,-0.099509,0.007835,0.112021,-0.056769
2,15400,0.118950,0.142323,-0.117174,0.153851,-0.086589,0.058181,-0.027907,0.033773,-0.066041,...,-0.027494,0.055437,-0.110124,-0.063313,0.225932,0.111279,-0.045200,-0.061104,0.074366,-0.056196
3,17402,0.021435,-0.040574,0.122709,0.057783,-0.111673,0.015955,-0.046230,0.005016,-0.228196,...,-0.025130,0.004343,-0.045501,-0.061306,0.119307,0.004543,-0.248092,-0.084348,0.007011,-0.004507
4,17506,-0.074041,-0.031085,0.005534,0.041157,-0.051489,0.090248,-0.044207,-0.069160,-0.074115,...,0.026688,0.011531,-0.011733,0.056475,0.113569,0.022809,-0.226393,0.078437,-0.031214,-0.109091


In [11]:
w2v.to_csv(os.path.join('data','clean',f'2020-08-24-listing-description-w2v-{vsize}.csv.gz'), index=False)